In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *

from xfem import *
from xfem.mlset import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

DrawDC = MakeDiscontinuousDraw(Draw)

importing ngsxfem-2.1.2504


In [2]:
# physical parameters for Biot
mu  = 1e6
lam = 1e6
# alpha = 1
alpha = 1
K = 1e2 # k^-1
nu = 1
# s0 = 10
s0 = 1e-2

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [24]:
# DG space order
order_eta = 2
order_u = 2
order_p = 1
# penalty parameters
beta_eta = 400
beta_u = 300
# ghost penalty parameters
gamma_s = 0.1
gamma_u = 0.1
gamma_p = 0.5
gamma_m = 0
tau_p = 0

In [25]:
eta_x = 0
eta_y = 0
u_x = 0
u_y = IfPos(y-0.5,x*(x-1),-x*(x-1))
exact_eta = CF((eta_x, eta_y))
exact_u = CF((u_x,u_y))

# 向量形式
fe = CF((0, 0))
fm = CF((0, 0))
fp = 0
etaD = exact_eta
uD = exact_u
uD2 = CF((0, 0))

In [26]:
# Construct background mesh
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
# h0 = 1/32
h0 = 0.02
# Geometry and Mesh
square = SplineGeometry()
square.AddRectangle((0, 0), (1, 1), bcs=('b','r','t','l'))
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# Draw(mesh)

In [27]:
line1 = y - x - 0.1
line2 = y - x + 0.1
line3 = y + x - 0.5
line4 = y + x - 1.5
level_sets = (line1,line2,line3,line4)
nr_ls = len(level_sets)
lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

for i, lset_p1 in enumerate(lsets_p1):
    InterpolateToP1(level_sets[i],lset_p1)
omega = ~DomainTypeArray((NEG,POS,POS,NEG), lsets=lsets_p1, persistent_compress=True)
boundary = omega.Boundary()

In [28]:
mlci = MultiLevelsetCutInfo(mesh,lsets_p1)
els_hasneg = mlci.GetElementsWithContribution(omega)
els_if = mlci.GetElementsWithContribution(boundary)
facets_gp = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_if,use_and=True)
interior_facets = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_hasneg,bnd_val_a=False,bnd_val_b=False,use_and=True)

Draw(BitArrayCF(els_hasneg),mesh)
# Draw(BitArrayCF(els_if),mesh)

els_if_single = {}
for i,dtt in enumerate(boundary):
    els_if_single[dtt] = mlci.GetElementsWithContribution(dtt)
    # Draw(BitArrayCF(els_if_single[dtt]),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [29]:
#Unfitted DG spaces
Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
# Phbase = L2(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
E = Compress(Ehbase, GetDofsOfElements(Ehbase, els_hasneg))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, els_hasneg))
P = Compress(Phbase, GetDofsOfElements(Phbase, els_hasneg))
fes = E*U*P
(eta,u,p), (kxi,v,q) = fes.TnT()

In [30]:
h = specialcf.mesh_size
ne = specialcf.normal(mesh.dim) # normal vectors on faces
normals = omega.GetOuterNormals(lsets_p1) # levelsets的外法向量

domega = dCut(lsets_p1,omega,definedonelements=els_hasneg)
# dk = dCut(lsets_p1, omega, skeleton=True, definedonelements=interior_facets)
dk = dx(skeleton=True,definedonelements=interior_facets)
dw = dFacetPatch(definedonelements=facets_gp) # ghost penalty

# Gamma
dsc = {dtt:dCut(lsets_p1,dtt,definedonelements=els_if_single[dtt]) for dtt in boundary}

# outer boundaries for eta
dseta = ds(skeleton=True,definedon=mesh.Boundaries('l|r|t|b'))

# outer boundaries for u
dsn = ds(skeleton=True,definedon=mesh.Boundaries('l|r')) #zero boundary
dsu = ds(skeleton=True,definedon=mesh.Boundaries('t|b')) # inflow boundary

In [31]:
strain_eta = Sym(Grad(eta))
strain_kxi = Sym(Grad(kxi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*ne
mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*ne
jump_eta = eta - eta.Other()
jump_kxi = kxi - kxi.Other()


jump_p = p - p.Other()
jump_q = q - q.Other()
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())


mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*ne
mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [32]:
Ah = BilinearForm(fes)
############## Eq1 ##############
# Ae
Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
        - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta))*dk \
        - (InnerProduct(Stress(Sym(Grad(eta)))*ne,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*ne,eta))*dseta

Ah +=  beta_eta/h*(2*mu*InnerProduct(jump_eta,jump_kxi)+lam*InnerProduct(jump_eta,ne)*InnerProduct(jump_kxi,ne)) * dk
Ah += beta_eta/h*(2*mu*InnerProduct(eta,kxi) + lam*InnerProduct(eta,ne)*InnerProduct(kxi,ne))*dseta

# Be
Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dk - p*kxi*ne*dseta)

# ghost penalty for eta
Ah += gamma_s / (h**2) * (eta - eta.Other()) * (kxi - kxi.Other()) * dw

############## Eq2 ############## 
# Am
Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
        - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        + K*InnerProduct(u,v)*domega
Ah += - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*InnerProduct(u,v))*dsu # Gamma_p

un = InnerProduct(u, ne)
vn = InnerProduct(v, ne)
Ah += - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*un*vn)*dsn # Gamma_no



 # Bm 
Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*ne*dsn + p*v*ne*dsu

# ghost penalty for velocity
Ah += gamma_u / (h**2) * (u - u.Other()) * (v - v.Other()) * dw

############## Eq3 ##############

# M
Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# -Be
Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dk - q*eta*ne*dseta)

# -Bm
Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*ne*dsn - q*u*ne*dsu


# ghost penalty for pressure 
Ah += gamma_p * (h**3) * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dk

# stabilization
Ah += tau_p*h*jump_p*jump_q*dk
Ah.Assemble()

In [33]:
# r.h.s
lh = LinearForm(fes) 
############## Eq1 ##############
lh += fe*kxi*domega - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*ne)*dseta 
lh += beta_eta/h*(2*mu*InnerProduct(etaD,kxi) + lam*InnerProduct(etaD,ne)*InnerProduct(kxi,ne))*dseta

############## Eq2 ##############
lh += fm*v*domega 
lh += - (nu*InnerProduct(uD,Grad(v)*ne) - nu*beta_u/h*uD*v)*dsu
lh += - (nu*InnerProduct(uD2,Grad(v)*ne) - nu*beta_u/h*InnerProduct(uD2,ne)*InnerProduct(v,ne))*dsn

############## Eq3 ##############
lh += fp*q*domega 
# lh += - alpha*q*etaD*ne*dseta - q*uD2*ne*dsn - q*uD*ne*dsu
lh += - alpha*q*etaD*ne*dseta - q*uD*ne*dsu
lh.Assemble()

In [34]:
gfu = GridFunction(fes)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

In [35]:
mask = omega.Indicator(lsets_p1)
Draw(mask*gfu.components[0],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [36]:
Draw(mask*gfu.components[1],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [37]:
Draw(mask*gfu.components[2],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
vtk = VTKOutput(mesh,[gfu.components[0],gfu.components[1],gfu.components[2]],['eta','u','p'],"/mnt/d/ngs_output/Biot_Brinkman/Ex_sliver",subdivision=0)
vtk.Do() 